In [141]:
!pip install pymorphy2

In [142]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2020-10-23 05:41:13--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.241, 5.45.205.244, 5.45.205.243, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.241|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskmar08.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-10-23 05:41:13--  http://cache-mskmar08.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskmar08.cdn.yandex.net (cache-mskmar08.cdn.yandex.net)... 5.45.222.32, 2a02:6b8:0:2b03::e8
Connecting to cache-mskmar08.cdn.yandex.net (cache-mskmar08.cdn.yandex.net)|5.45.222.32|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.3’

mystem-3.0-linux3.1 100%[================

In [143]:
import re
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
import csv
import sqlite3
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
m = Mystem()
morph = MorphAnalyzer()
from string import punctuation
from tqdm.auto import tqdm
from nltk import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Датафрейм
Мы скачали слишком много данных, поэтмоу взяли 700 и убрали повторяющиеся новости (иногда были)

In [144]:
df = pd.read_csv('data_of_news_p.csv', sep=',')

In [145]:
df = df[:700]
df

,Unnamed: 0,newspaper,tags,headers,articles,links
0,0,Panorama,Общество,В т о р г о в о й с е т и П я т ё р о ч ...,В торговой сети Пятёрочка в продаже появились ...,https://panorama.pub/50439-v-torgovoj-seti-pya...
1,1,Panorama,Общество,И л ь ю В а р л а м о в а о б в и н и л и ...,Мэрия Воркуты обратилась с иском к видеоблогер...,https://panorama.pub/50505-varlamova-obvinili....
2,2,Panorama,Общество,В Р о с с и и в в е д у т о б я з а т е ...,Комитет Государственной думы по бюджету и нало...,https://panorama.pub/50500-nalog-na-wi-fi-rout...
3,3,Panorama,Общество,М и к у Х а ц у н э и с п о л н и т р е ...,Японская виртуальная певица Мику Хацуне на гот...,https://panorama.pub/50494-miku-hatsune-ispoln...
4,4,Panorama,Общество,L о т м е н я е т т р е б о в а н и е о ...,Крупнейший авиаперевозчик Германии L объявил о...,https://panorama.pub/50502-lufthansa-otmenyaet...
...,...,...,...,...,...,...
695,695,Panorama,Общество,В К а л и н и н г р а д е с н е с у т п ...,Мэрия Калининграда дала согласие на снос монум...,https://panorama.pub/43492-v-kaliningrade-snes...
696,696,Panorama,Общество,В К у р с к е а р е с т о в а л и т а м ...,Сотрудниками правоохранительных органов Курско...,https://panorama.pub/43484-arestovali-tamozhen...
697,697,Panorama,Общество,"С у д е й , у л и ч ё н н ы х в о в з я ...",Госдума приняла в первом чтении законопроект О...,https://panorama.pub/43454-sudej-osvobodjat.html
698,698,Panorama,Общество,П о с л е л е к ц и и Г о р б а ч ё в а ...,Неприятно завершилась лекция в одной из аудито...,https://panorama.pub/43436-posle-lektsii-gorba...


In [146]:
df = df.drop_duplicates(subset ="links", 
                     keep = False, ignore_index=True) 

In [147]:
df

,Unnamed: 0,newspaper,tags,headers,articles,links
0,342,Panorama,Общество,"S I р а з р а б о т а е т к о м п а с , ...",Итальянский производитель одежды премиумкласса...,https://panorama.pub/48565-stone-island-razrab...
1,343,Panorama,Общество,И з з а с д в и г а л и т о с ф е р н ы х ...,Военные действия Армении и Азербайджана спрово...,https://panorama.pub/48602-nagornyj-karabah-vo...
2,344,Panorama,Общество,Л у к а ш е н к о п о т р е б о в а л и с ...,Александр Лукашенко выступая перед президиумом...,https://panorama.pub/48538-lukashenko-potrebov...
3,345,Panorama,Общество,М о с к в а в в о д и т с и с т е м у б ...,В связи с осложнившейся эпидемиологической обс...,https://panorama.pub/48561-moskva-vvodit-siste...
4,346,Panorama,Общество,Ч е л я б и н ц ы к м н е с л и н а ...,"Необычный случай произошел на трассе М, соедин...",https://panorama.pub/48509-chelyabintsy.html
...,...,...,...,...,...,...
353,695,Panorama,Общество,В К а л и н и н г р а д е с н е с у т п ...,Мэрия Калининграда дала согласие на снос монум...,https://panorama.pub/43492-v-kaliningrade-snes...
354,696,Panorama,Общество,В К у р с к е а р е с т о в а л и т а м ...,Сотрудниками правоохранительных органов Курско...,https://panorama.pub/43484-arestovali-tamozhen...
355,697,Panorama,Общество,"С у д е й , у л и ч ё н н ы х в о в з я ...",Госдума приняла в первом чтении законопроект О...,https://panorama.pub/43454-sudej-osvobodjat.html
356,698,Panorama,Общество,П о с л е л е к ц и и Г о р б а ч ё в а ...,Неприятно завершилась лекция в одной из аудито...,https://panorama.pub/43436-posle-lektsii-gorba...


# Предобработка и служебные функции

In [148]:
raw_texts = []
for row in df['articles']:
    raw_texts.append(row)

In [149]:
from string import punctuation
def clean(text):
  for i in punctuation + '—«»':
    text = text.replace(i, '')
  text = text.replace('  ',' ')
  return text

In [150]:
def lemmatize(word):
  ana = m.analyze(word.lower())
  for word in ana:
    if 'analysis' in word:
      if len(word['analysis']) != 0:
        lex = word['analysis'][0]['lex']
      else:
        lex = word['text']
  return lex

In [151]:
def make_sent(sent):
  s = ''
  for i in sent:
    s += i + ' '
  return s[:-1]

In [152]:
def sent_lemmatize(sent):
  pos = []
  lemm = []
  lex = []
  lemm_sent = []
  for i in m.analyze(clean(sent)):
    if 'analysis' in i:
      if i['analysis'] == []:
        pos.append('UNDFND')
        lex.append(i['text'])
        lemm.append(i['text'])
      else:
        lex.append(i['text'].lower())
        lemm.append(i['analysis'][0]['lex'])
        pos.append(re.search(r'([A-Z])', i['analysis'][0]['gr']).group())
    elif (i['text'] != ' ' and i['text'] != '\n'):
      pos.append('UNDFND')
      lex.append(i['text'].lower())
      lemm.append(i['text'])
  return lex, lemm, pos

In [153]:
texts_pos = []
texts_lem = []
texts_lex = []
texts_sent = []
texts_lemmatized_sent = []
for text in tqdm(raw_texts):
  lem_sents = []
  sents = []
  lex, lemm, pos = sent_lemmatize(text)
  texts_lex.append(lex)
  texts_pos.append(pos)
  texts_lem.append(lemm)
  for i in nltk.sent_tokenize(text):
    sents.append(i)
    lem_sents.append(sent_lemmatize(i)[1])
  texts_sent.append(sents)
  texts_lemmatized_sent.append(lem_sents)

# Рабочий датафрейм

In [154]:
texts = pd.DataFrame()
texts['title'] = df.headers
texts['text'] = df.articles
texts['sentences'] = texts_sent
texts['lemmatized sentences'] = texts_lemmatized_sent
texts['lexems'] = texts_lex 
texts['lemms'] = texts_lem
texts['tags'] = texts_pos
texts['links'] = df.links
texts

,title,text,sentences,lemmatized sentences,lexems,lemms,tags,links
0,"S I р а з р а б о т а е т к о м п а с , ...",Итальянский производитель одежды премиумкласса...,[Итальянский производитель одежды премиумкласс...,"[[итальянский, производитель, одежда, премиумк...","[итальянский, производитель, одежды, премиумкл...","[итальянский, производитель, одежда, премиумкл...","[A, S, S, S, UNDFND, UNDFND, P, A, S, V, S, V,...",https://panorama.pub/48565-stone-island-razrab...
1,И з з а с д в и г а л и т о с ф е р н ы х ...,Военные действия Армении и Азербайджана спрово...,[Военные действия Армении и Азербайджана спров...,"[[военный, действие, армения, и, азербайджан, ...","[военные, действия, армении, и, азербайджана, ...","[военный, действие, армения, и, азербайджан, с...","[A, S, S, C, S, V, A, A, S, P, S, A, S, S, S, ...",https://panorama.pub/48602-nagornyj-karabah-vo...
2,Л у к а ш е н к о п о т р е б о в а л и с ...,Александр Лукашенко выступая перед президиумом...,[Александр Лукашенко выступая перед президиумо...,"[[александр, лукашенко, выступать, перед, през...","[александр, лукашенко, выступая, перед, презид...","[александр, лукашенко, выступать, перед, прези...","[S, S, V, P, S, A, S, S, S, V, V, S, P, S, A, ...",https://panorama.pub/48538-lukashenko-potrebov...
3,М о с к в а в в о д и т с и с т е м у б ...,В связи с осложнившейся эпидемиологической обс...,[В связи с осложнившейся эпидемиологической об...,"[[в, связь, с, осложняться, эпидемиологический...","[в, связи, с, осложнившейся, эпидемиологическо...","[в, связь, с, осложняться, эпидемиологический,...","[P, S, P, V, A, S, S, S, V, S, P, A, S, S, P, ...",https://panorama.pub/48561-moskva-vvodit-siste...
4,Ч е л я б и н ц ы к м н е с л и н а ...,"Необычный случай произошел на трассе М, соедин...","[Необычный случай произошел на трассе М, соеди...","[[необычный, случай, происходить, на, трасса, ...","[необычный, случай, произошел, на, трассе, м, ...","[необычный, случай, происходить, на, трасса, м...","[A, S, V, P, S, S, V, S, C, S, S, S, A, A, S, ...",https://panorama.pub/48509-chelyabintsy.html
...,...,...,...,...,...,...,...,...
353,В К а л и н и н г р а д е с н е с у т п ...,Мэрия Калининграда дала согласие на снос монум...,[Мэрия Калининграда дала согласие на снос мону...,"[[мэрия, калининград, давать, согласие, на, сн...","[мэрия, калининграда, дала, согласие, на, снос...","[мэрия, калининград, давать, согласие, на, сно...","[S, S, V, S, P, S, S, S, S, V, S, A, S, S, V, ...",https://panorama.pub/43492-v-kaliningrade-snes...
354,В К у р с к е а р е с т о в а л и т а м ...,Сотрудниками правоохранительных органов Курско...,[Сотрудниками правоохранительных органов Курск...,"[[сотрудник, правоохранительный, орган, курски...","[сотрудниками, правоохранительных, органов, ку...","[сотрудник, правоохранительный, орган, курский...","[S, A, S, A, S, V, V, S, V, A, S, V, P, S, V, ...",https://panorama.pub/43484-arestovali-tamozhen...
355,"С у д е й , у л и ч ё н н ы х в о в з я ...",Госдума приняла в первом чтении законопроект О...,[Госдума приняла в первом чтении законопроект ...,"[[госдума, принимать, в, первый, чтение, закон...","[госдума, приняла, в, первом, чтении, законопр...","[госдума, принимать, в, первый, чтение, законо...","[S, V, P, A, S, S, P, S, S, P, S, A, S, S, V, ...",https://panorama.pub/43454-sudej-osvobodjat.html
356,П о с л е л е к ц и и Г о р б а ч ё в а ...,Неприятно завершилась лекция в одной из аудито...,[Неприятно завершилась лекция в одной из аудит...,"[[неприятно, завершаться, лекция, в, один, из,...","[неприятно, завершилась, лекция, в, одной, из,...","[неприятно, завершаться, лекция, в, один, из, ...","[A, V, S, P, A, P, S, A, S, P, S, S, S, S, S, ...",https://panorama.pub/43436-posle-lektsii-gorba...


In [181]:
pos_tags = ['A', 'ADV', 'ADVPRO', 'ANUM', 'APRO', 'COM', 'CONJ', 'INTJ', 'NUM', 'PART', 'PR', 'S', 'SPRO', 'V', 'P']

In [182]:
def pos(smth):
  words = []
  for i, tag in enumerate(texts['tags']):
    for t in range(len(tag)):
      if tag[t]==smth:
        words.append(texts['lexems'][i][t])
      else:
        words.append('')
  return words #списочек с подходящими словами

In [183]:
def lexx(smth):
  smth = smth.replace('"', '')
  words = []
  for i, lexem in enumerate(texts['lexems']):
    for l in texts['lexems'][i]:
      if l==smth:
        words.append(l)
      else:
        words.append('')
  return words

In [184]:
def lexpos(smth):
  splitted_smth = smth.split('+')
  words = []
  for i, tag in enumerate(texts['tags']):
    for t in range(len(tag)):
      if tag[t]==splitted_smth[1] and texts['lexems'][i][t]==splitted_smth[0]:
        words.append(texts['lexems'][i][t])
      else:
        words.append('')
  return words

In [185]:
def lem(smth):
  lemmatized_smth = lemmatize(smth)
  words = []
  for i, lem in enumerate(texts['lemms']):
    for t in range(len(lem)):
      if lem[t]==lemmatized_smth:
        words.append(texts['lexems'][i][t])
      else:
        words.append('')
  return words

In [186]:
def search_by_ngramm(start_req):
  req = start_req.split(' ')
  for_work = []
  finish = [] #тут словосочетания списками
  new = [] #это для сдвигов
  sent_of_cuts = [] #для кусков
  sent_of_sents = [] #для предложений
  words_lex = []
  links = []
  for i in req:
    if i in pos_tags:
      words = pos(i)
    elif i[0]=='"':
      words = lexx(i)
    elif '+' in i:
      words = lexpos(i)
    else:
      words = lem(i)
    for_work.append(words)

  for m in range(len(for_work)):
    sdvig = for_work[m][m:]
    sdvig.extend(for_work[m][:m])
    new.append(sdvig)
  for n in range(len(new[0])):
    cut = []
    for elem in new:
      cut.append(elem[n])
    if '' not in cut:
      finish.append(cut)
  for t in finish:
    sent_of_cuts.append(' '.join(t))
  final_sents = []
  for i, text in enumerate(texts['sentences']):
    for j, sent in enumerate(text):
      for s in sent_of_cuts:
        if s in ' '.join(clean(sent.lower()).split()):
          if sent not in final_sents:
            final_sents.append(sent)
            links.append(texts['links'].to_list()[i])
      
  return final_sents, links  

In [191]:
sents, links = search_by_ngramm('A A A A A')

In [192]:
for i in range(len(sents)):
  print(sents[i], '→ Читать продолжение в источнике... →', links[i])
  print()

Таким был Иисус Христос, таким до него были Зевс, Озирис, Мардук, Фамуз, а также куда менее известный благостный бог Вицлипуцли, которого некогда почитали ацтеки в Мексике. → Читать продолжение в источнике... → https://panorama.pub/48521-doch-govarda-lavkrafta.html

